<a href="https://colab.research.google.com/github/HenryLaur/ml-projects/blob/main/huggingface/case_law_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### COLAB URL
https://colab.research.google.com/drive/1HaFH0tox1W9VYZCQPvigZQdXDYAN9CES?usp=sharing

In [ ]:
!pip install sentence-transformers
!pip install langchain
!pip install pinecone-client
!pip install accelerate
!pip install bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.4 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=29dd770921dd7c56f3d26a7669332c9a72d037dc2e2dc09ef4920a0dccdc7688
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 794.3/794.3 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 44.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.4/179.4 kB 3.0 MB

### Load data to colab and unzip

Did it in Kaggle but kaggle didnt seem to have a good way of sharing with just link without making it completly public

In [ ]:
!unzip archive.zip

In [ ]:
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
import pinecone
from tqdm.auto import tqdm
from uuid import uuid4
from langchain.vectorstores import Pinecone
from transformers import pipeline
from langchain.llms import HuggingFacePipeline
import torch
from langchain.chains import RetrievalQA
import transformers
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

## Create our embedding transformer

### Used mini model for testing
all-MiniLM-L6-v2

### Used model for inference
all-mpnet-base-v2

In [ ]:
model_encoder = SentenceTransformer('all-mpnet-base-v2', device='cuda')

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

### We cant upload all the data at once, so lets create a dataset so we can load data to pinecone in batches

In [ ]:
import os
from torch.utils.data import Dataset

class LawDataset(Dataset):
    def __init__(self, root_dirs):
        self.root_dirs = root_dirs
        # We can have many root dirs so rather than keeping only file names lets keep full paths
        self.list_file = [os.path.join(root_dir, file) for root_dir in root_dirs for file in os.listdir(root_dir)]

    def __len__(self):
        return len(self.list_file)


    def __getitem__(self, index):
        img_file = self.list_file[index]
        return TextLoader(img_file).load()


### Batch size needs to be low enough that we can load all the data in the batch to pinecone, limit is about 2mb

In [ ]:
from torch.utils.data import DataLoader
data = LawDataset(["/content/Object_statutes", "/content/Object_casedocs"])
batch_size = 5
train_dataloader = DataLoader(data, batch_size=batch_size ,collate_fn=lambda text: text)


In [ ]:
len(data)

3111

### Lets test that our embedding encoder works

In [ ]:
res = model_encoder.encode("Test Content")

### Init pinecone

In [ ]:
from google.colab import userdata

index_name = 'law-cases'
pinecone.init(
    api_key=userdata.get('pinecone'), # Add your own api key
    environment="gcp-starter"
)

if index_name not in pinecone.list_indexes():
    pinecone.create_index(
        name=index_name,
        metric='cosine',
        dimension=res.shape[0]
    )
index = pinecone.Index(index_name)

index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.89066,
 'namespaces': {'': {'vector_count': 89066}},
 'total_vector_count': 89066}

### Now we will loop through the dataloader and upload each batch to pinecone. We also add text to metadata for later

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
text_field = "text"

for i, records in enumerate(tqdm(train_dataloader)):
    # we want a flat list
    records = [item for row in records for item in row]
    record_texts = text_splitter.split_documents(records)
    record_metadatas = [{
        "chunk": j, text_field: text.page_content, 'source': text.metadata['source']
    } for j, text in enumerate(record_texts)]

    ids = [str(uuid4()) for _ in range(len(record_texts))]
    embeds = model_encoder.encode([text.page_content for text in record_texts])

    index.upsert(vectors=zip(ids, embeds.tolist(), record_metadatas))


  0%|          | 0/623 [00:00<?, ?it/s]

### Now to make sure that our database is populated

In [ ]:
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.89066,
 'namespaces': {'': {'vector_count': 89066}},
 'total_vector_count': 89066}

### The next steps want our model to return a python list not a numpy array so we can create a simple wrapper function

In [ ]:
def model_encode_list(text):
    return model_encoder.encode(text).tolist()

In [ ]:
vectorstore = Pinecone(
    index, model_encode_list, text_field
)

/usr/local/lib/python3.10/dist-packages/langchain_community/vectorstores/pinecone.py:62: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


### Test that our query will return similar documents

In [ ]:
query = "The appellant on February 9, 1961 was appointed as an Officer in Grade III in the respondent Bank ( for short 'the Bank'). He was promoted on April 1, 1968 to the Grade officer in the Foreign Exchange Department in the Head Office of the Bank. Sometime in 1964, MCH Society ( for short 'the Society') was formed of which the appellant was one of the chief promoters and thereafter its Secretary. The object of the Society was to construct residential premises for the employees of the Bank and its other members. It appears that the complaint was received in respect of the affairs of the Society relating to misappropriation of the funds of the Society and consequently, in exercise of the powers under Section S of Act A1, the Registrar on April 23, 1969 instituted an inquiry thereof. P1 was appointed the Registrar's nominee who on October 4, 1969; submitted the report holding the appellant and two other office bearers of the Society negligent in dealing with the funds of the Society causing a loss to the tune of Rs. 3,59,000/-. The Registrar on October 21, 1969, passed an order appointing an officer under Section S of A1 to assess the loss caused to the Society. However, the Government by its order dated November 29, 1969 annulled the Registrar's order dated April 23, 1969 and October 21, 1969 and directed a fresh inquiry into the affairs of the Society. On December 17, 1969, the Bank issued show cause notice to the appellant to explain within fifteen days his alleged negligent conduct in dealing with the affairs of the Society as revealed in the report dated 4th October, 1969. In the meantime, P2 came to be appointed by the Registrar vide his order dated 26th July, 1969, to make inquiries under Section S of A1. Petitioner by his reply dated 18/22th January, 1970 submitted his explanation and also challenged the legality of the inquiry and the findings recorded therein. On 5th March, 1970, P3, treasurer of the Society and an employee of the Bank criminal complaints in the Court of Addl. Chief Presidency Magistrate  alleging that the appellant and two other office bearers of the society had dishonestly misappropriated a sum of Rs. 51,000/ and Rs. 80,000/- respectively which was entrusted to the appellant in his capacity as Promoter and Secretary of the Society and thereby committed criminal breach of trust. The Magistrate framed the charges against the appellant. The Bank having regard to the serious misconduct of the appellant involving moral turpitude vide its order dated 3rd November, 1970 suspended the appellant pending trial. The appellant protested this action of the Bank complaining that he was not given an opportunity of hearing before passing the order of suspension. In the meantime, P2, the authorized officer appointed by the Registrar vide his order dated 9th October, 1971 held the appellant liable to pay Rs. 2,36,000/- to the Society in addition to the amount of Rs. 2,03,000/- for which he (the appellant) and two other office bearers of the Society were held jointly liable. The Bank in view of this finding, vide its order dated 29th November, 1971 terminated the services of the appellant with effect from 1st December, 1971 along with notice pay. The appellant protested against the Action of the Bank and on 3rd December, 1971 filed detailed representation against the order of termination. The Bank replied to the appellant's representation and justified its action. The appellant on 28th December, 1971 submitted his reply to the Bank stating, inter alia, that the termination of his services was not simplicitor and was in violation of the principles of natural justice; that no opportunity of hearing was given to him; that the termination order attached stigma. The appellant aggrieved by the findings and order made by P2 appealed before Tribunal. In the meantime, the criminal proceedings ended in conviction vide order dated 27th March, 1972 passed by the Addl. Chief Metropolitan Magistrate. The appellant challenged the order of conviction and sentence in the High Court  and during the pendency of the said appeal, the Tribunal vide its order dated April 12, 1973 dismissed the appellant's appeal but reduced the liability by Rs. 72,000/-. On November 12, 1973, the High Court allowed the criminal appeal and acquitted the appellant. The High Court, however, in its order observed that since the services of the appellant were terminated in view of the criminal proceedings and since the appellant has been acquitted, representation, if a any, by the appellant to the Bank for reinstatement may be considered sympathetically. Taking clue from the observations made by the High Court, the appellant filed three representations, the last being dated 3rd May, 1975 requesting the Bank to revoke the order of termination and be reinstated. The Bank vide its communication dated May 21, 1975 refused to reinstate the appellant. The appellant, therefore, on July 23, 1975 filed the writ petition in the High Court for quashing the orders dated 29th November 1971, 27th December, 1971 and 21st May, 1975 passed by the Bank. The learned Single Judge of the High Court by his judgment and order dated December 6/7, 1979 granted desired relief to the appellant. The Bank aggrieved by the judgment and order passed by the learned Single Judge preferred an appeal under Clause 15 of the Letters Patent. The appeal was heard by the Division Bench. The Division Bench of the High Court did not agree with the judgment passed by the learned Single Judge and consequently by its judgment and order dated April 16 1985 allowed the appeal and dismissed the writ petition the ground of laches and also on merits. It is this judgment and order of the High Court which is impugned in this appeal."

vectorstore.similarity_search(
    query,
    k=3
)

[Document(page_content="1.  The petitioner joined service of the State Bank of Patiala as a Clerk in 1968 and was promoted to the post of Trainee Officer with effect from August 1, 1974. By letter No. Staff/Genl/3749 dated August 1, 1974 the petitioner was informed that the Bank's Board of Directors had approved his promotion as Trainee Officer under Group B Grade II with effect from August 1, 1974 in the scale of Rs 500-1030. By a subsequent letter dated August 14, 1975 he was informed that the Board of Directors had confirmed him as Officer Grade II with effect from August 1, 1975 in the said scale. When the petitioner was working as Branch Manager at Birhana Road, Kanpur, he is alleged to have committed some misdemeanour. By a letter dated January 9, 1980 he was called upon to explain the purchase bills in excess of his discretionary power of Rs 1 lakh. The petitioner sent his reply to the said memorandum on January 31, 1980 contending that the purchase of the bills was within his d

### Create our LLM

using google/flan-t5-xl because its light and fast. Tried a few more powerful models like upstage/SOLAR-10.7B-Instruct-v1.0 but they took extreme long to inference.

In [ ]:
model_name = "google/flan-t5-xl"
tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForSeq2SeqLM.from_pretrained(model_name, load_in_8bit=True, device_map='auto')

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.44k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/53.0k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.45G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

### Setup our pipeline and retrivalQA

In [ ]:
pipe = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=512,
)

local_llm = HuggingFacePipeline(pipeline=pipe)

In [ ]:
qa = RetrievalQA.from_chain_type(
    llm=local_llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
    return_source_documents=True
)

In [ ]:
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

Note: google/flan-t5-xl can actually take in sequences longer than 512 so we can ignore the warning. However when replacing google/flan-t5-xl with another model the sequence length should probably be around 4k because the questions are so long

In [ ]:
process_llm_response(qa(query))

The appellant was one of the chief promoters and thereafter its Secretary.

Sources:
/content/Object_casedocs/C1523.txt
/content/Object_casedocs/C1240.txt
/content/Object_casedocs/C1523.txt
/content/Object_casedocs/C1405.txt


In [ ]:
query = "This criminal appeal is directed against the judgment of the High Court dismissing the appeal but modifying the sentence. The appellant took his trial on the allegations that he had dishonestly and fraudulently misappropriated a sum of Rs. 3851.60, which amount was in his control in the capacity of a public servant, i.e. Cashier-cum-Accountant in the office of the District Veterinary Officer and that he had wilfully and with the intent to defraud altered and even mutilated the cash book (Ex. 4) and also had forged the said cash book with the intent to defraud the government to the extent of the aforesaid amount. The trial court accepting the case of the prosecution convicted the appellant under all the charges and sentenced him to rigorous imprisonment for a period of one year and to pay a fine of Rs. 3900, in default to suffer rigorous imprisonment for one year and also to undergo rigorous imprisonment for one year on each of the convictions in addition to pay a fine of Rs. 200, in default to suffer imprisonment for a period of two months. On appeal, the High Court while confirming the judgment of the trial court in its entirety, reduced the fine amount from Rs. 3900 to Rs. 2000, in default to undergo six months rigorous imprisonment for the conviction. In other respects, the sentences awarded for the convictions under the other two charges were confirmed. Hence this appeal."
process_llm_response(qa(query))

The appellant

Sources:
/content/Object_casedocs/C414.txt
/content/Object_casedocs/C1215.txt
/content/Object_casedocs/C2798.txt
/content/Object_casedocs/C562.txt


In [ ]:
query = 'The four appellants, along with P1 son of P2, were jointly tried in the court of Additional Sessions Judge on the following charges: "That you all accused nos. 1 to 5 on or about 12th day of November, 1967 at about 5-45 a.m. near XYZ Road, formed an unlawful assembly and in prosecution of the common object of such assembly viz. : to commit murder of complainant P3 or in order to cause murder of P3 or grievous hurts to him committed the offence of rioting and thereby committed an offence punishable under Section 147 of the Indian Penal Code and within the cognizance of this Court. That you all on the same date, time and place, were members of unlawful assembly, in prosecution of common object of which viz. : to commit murder of P3 or to cause grievous hurt to him, one or all you caused grievous hurts to him which offence you knew to be likely to be committed in prosecution of the common object of the said assembly. That you all on the same date, time and place attempted to cause murder of P3 Deshmukh, in furtherance of common intention and thereby committed an offence punishable." The order of the trial court convicting them all concludes thus: "All the five accused are convicted for the offence of rioting punishable and each is sentenced to rigorous imprisonment for the period of six months and to a fine of Rs. 501-, in default, rigorous imprisonment for two weeks for that offence. Accused shall surrender to their bail." 2. They all jointly appealed to the High Court by one memorandum of appeal. Learned judge admitted the appeal only on behalf of P1 and dismissed in limine the appeal on behalf of the four -appellants before us. The only point which concerns this Court in the present appeal by special leave relates to the correctness of the order dismissing in limine the appeal on behalf of the four appellants, when the appeal on behalf of P1, co-accused was admitted for hearing on the merits after notice to, the State.'
process_llm_response(qa(query))

P1

Sources:
/content/Object_casedocs/C2085.txt
/content/Object_casedocs/C2526.txt
/content/Object_casedocs/C380.txt
/content/Object_casedocs/C1993.txt


### Dataset Question Results

When looking at the fetched sources then they dont line up with the provided answers in the dataset. However when looking at the question and provided source then there are quite a few similarities meaning that it still tries to find the best matching document.

Lets try our custom query.

In [ ]:
query = "When was the appellant appointed as Auction Recorder?"
process_llm_response(qa(query))

19.11.1986

Sources:
/content/Object_casedocs/C2343.txt
/content/Object_casedocs/C1300.txt
/content/Object_casedocs/C3.txt
/content/Object_casedocs/C308.txt


Answer is correct (taken from C3.txt point 2) and we can see that the document is also in the sources. Overall we can see that the model is working

## Further improvment ideas

### Data exploration
Look at the provided data e.g used words, frequencies, sentence lengths, ngrams, word clouds. That can give ideas how to better use the data or preprocess it.

### Pretrained/finetuned models
The data is focused on law so finetuning a model on other similar documents will give slight improvements

### Chuck size
Since the questions are very long the answers might be in separate chucks meaning the "correct" chucks might not always be selected. Finding the optimum chuck size will give better results